In [1]:
import pandas as pd 
import numpy as np
from tqdm import *
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pickle
%matplotlib inline
import os
import sys
from datetime import *
pd.set_option('display.max_columns', 500)
import time
import math

In [2]:
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename

def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

def metric_fun(pred , label):
    return math.exp(-100*abs(label - pred)/max(abs(label),10**(-15)))

In [3]:
def get_data(i):
    """
    get_ori_data
    """
    start = time.clock()
    data = pd.read_csv('../data/dataset/'+str(i).zfill(3)+'/201807.csv',parse_dates=[0])
    res = pd.read_csv('../data/template_submit_result.csv',parse_dates=[0])[['ts','wtid']]
    
    res = res[res['wtid']==i]
    res['flag'] = 1
    data = res.merge(data, on=['wtid','ts'],how = 'outer')
    data = data.sort_values(['wtid','ts']).reset_index(drop = True)
    elapsed = (time.clock() - start)
    print("Time used:",elapsed)
    
    return data 

In [4]:
def resample_fe(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,1))
    except:
        df = Df[['ts',j]].copy()
        var = df[j].resample('1T').mean().reset_index()
        del df[j]
        minute = (df.ts.dt.minute.values/1).astype(int)*1
        time_list = list()
        for g,h,k in zip(df.ts.dt.day,df.ts.dt.hour,minute):
            time_list.append('2018-07-{} {}:{}:00'.format(g,h,k))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left').drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,1))
    return res

def resample_fe_median(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    except:
        df = Df[['ts',j]].copy()
        var = df.resample('1T').median().reset_index()
        del df[j]
        minute = (df.ts.dt.minute.values/1).astype(int)*1
        time_list = list()
        for g,h,k in zip(df.ts.dt.day,df.ts.dt.hour,minute):
            time_list.append('2018-07-{} {}:{}:00'.format(g,h,k))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left').drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    return res


def resample_fe_mode(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    except:
        df = Df[['ts',j]].copy()
        var = df[j].resample('2T').apply(lambda x:np.min(pd.Series.mode(x))).reset_index()
        del df[j]
        minute = (df.ts.dt.minute.values/2).astype(int)*2
        time_list = list()
        for g,h,k in zip(df.ts.dt.day,df.ts.dt.hour,minute):
            time_list.append('2018-07-{} {}:{}:00'.format(g,h,k))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left').drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    return res

def resample_fe_mode_mean(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    except:
        df = Df[['ts',j]].copy()
        var = df[j].resample('2T').apply(lambda x:np.nanmean(pd.Series.mode(x))).reset_index()
        del df[j]
        minute = (df.ts.dt.minute.values/2).astype(int)*2
        time_list = list()
        for g,h,k in zip(df.ts.dt.day,df.ts.dt.hour,minute):
            time_list.append('2018-07-{} {}:{}:00'.format(g,h,k))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left').drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}.pkl'.format(l,j,Type))
    return res

def resample_fe_mode_roll(Df,j,l,Type,w,mw):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}_{}_{}.pkl'.format(l,j,Type,w,mw))
    except:
        df = Df[[j]].copy()
        var = df[j].rolling(window = w , min_periods = mw,center = True).apply(lambda x:np.nanmin(pd.Series.mode(x)),raw=False).reset_index()
        del df[j]
        df_res = df.merge(var,on = 'ts',how = 'left')#.drop('ts',axis=1)
        res = df_res[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}_{}_{}.pkl'.format(l,j,Type,w,mw))
    return res

def resample_fe_mode_mean_roll(Df,j,l,Type,w,mw):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}_{}_{}.pkl'.format(l,j,Type,w,mw))
    except:
        df = Df[[j]].copy()
        var = df[j].rolling(window = w , min_periods = mw,center = True).apply(lambda x:np.nanmean(pd.Series.mode(x)),raw=False).reset_index()
        del df[j]
        df_res = df.merge(var,on = 'ts',how = 'left')#.drop('ts',axis=1)
        res = df_res[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}_{}_{}.pkl'.format(l,j,Type,w,mw))
    return res

def resample_fe_mode_111(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,111))
    except:
        df = Df[['ts',j]].copy()
        var = df[j].resample('1T').apply(lambda x:np.min(pd.Series.mode(x))).reset_index()
        del df[j]
        minute = (df.ts.dt.minute.values/1).astype(int)*1
        time_list = list()
        for g,h,k in zip(df.ts.dt.day,df.ts.dt.hour,minute):
            time_list.append('2018-07-{} {}:{}:00'.format(g,h,k))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left').drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,111))
    return res

def resample_fe_mode_1(Df,j,l,Type):
    try:
        res = load_variavle('../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,30))
    except:
        df = Df[['ts',j]].copy()
        var = df[j].resample('30S').apply(lambda x:np.min(pd.Series.mode(x))).reset_index()
        del df[j]
        second = (df.ts.dt.second.values/30).astype(int)*30
        time_list = list()
        for g,h,k,q in zip(df.ts.dt.day,df.ts.dt.hour,df.ts.dt.minute,second):
            time_list.append('2018-07-{} {}:{}:{}'.format(g,h,k,q))
        df['time'] = [pd.Timestamp(i) for i in time_list]
        var.columns = ['time',j]
        df = df.merge(var,on='time',how = 'left')#.drop('time',axis=1)
        res = df[j]
        save_variable(res,'../data/mode_fe/df_{}_{}_{}_{}.pkl'.format(l,j,Type,30))
    return res

In [5]:
# file_list = dict()
# for j in range(1,34):
#     data = get_data(j)
#     fe = [ i for i in data.columns if 'var' in i ]
#     zero_fe = list()
#     print(j,len(data))
#     for i in fe:
#         if i not in cat_fe + bool_fe :
#             try:
#                 if (data[i].value_counts()[data[i].mode()[0]]>20000)&(data[i].value_counts()[data[i].mode()[0]]<=60000):
# #                 if (data[i].value_counts()[data[i].mode()[0]]>len(data)*0.05)&(data[i].value_counts()[data[i].mode()[0]]<=len(data)*0.15):
#                     print(i,data[i].mode()[0],data[i].value_counts()[data[i].mode()[0]])
#                     zero_fe.append(i)
#             except:
#                 pass
#     file_list[j] = zero_fe

In [6]:
# save_variable(file_list,'../data/dict_cate_mode_set_2W.pkl')

In [8]:
cat_fe = ['var016','var020','var047']
bool_fe = ['var053','var066']
df = pd.DataFrame()
dict_cate = load_variavle('../data/dict_cate_mode_set_30W.pkl')
dict_cate_2 = load_variavle('../data/dict_cate_mode_set_25W.pkl')
dict_cate_3 = load_variavle('../data/dict_cate_mode_set_20W.pkl')
dict_cate_4 = load_variavle('../data/dict_cate_mode_set_15W.pkl')
dict_cate_5 = load_variavle('../data/dict_cate_mode_set_10W.pkl')
dict_cate_6 = load_variavle('../data/dict_cate_mode_set_6W.pkl')
dict_cate_7 = load_variavle('../data/dict_cate_mode_set_2W.pkl')

me_fe = ['var001','var002','var003','var004','var005','var006','var007',
         'var011','var012','var014','var015','var018','var021',
         'var022','var024','var027','var028','var029','var030','var031','var033',
         'var034','var035','var036','var038','var040','var042','var043',
         'var045','var046','var051','var052','var056','var057','var061','var067',
         'var068','var009','var017','var037','var048','var055','var062']

std_fe = ['var002','var006','var007','var011','var012','var014','var018',
          'var021','var022','var028','var029','var036','var037','var051','var055',
          'var057','var062','var067','var068']

for i in tqdm(range(1,34)):
    data = get_data(i)
    
    set_zero_fe = dict_cate[i]
    set_zero_fe_2 = dict_cate_2[i]
    set_zero_fe_3 = dict_cate_3[i]
    set_zero_fe_4 = dict_cate_4[i]
    set_zero_fe_5 = dict_cate_5[i]
    set_zero_fe_6 = dict_cate_6[i]
    set_zero_fe_7 = dict_cate_7[i]
#     set_near_fe = dict_cate_25[i]
    fe = [ i for i in data.columns if 'var' in i ]
    data.index = data.ts
    print("="*10+'_1_'+"="*10)
    print(set(fe)-set(set_zero_fe_7)-set(me_fe)-set(set_zero_fe)-set(set_zero_fe_2)-set(set_zero_fe_3)-set(set_zero_fe_4)-set(set_zero_fe_5)-set(set_zero_fe_6))
    print("="*10+'_set_zero_fe_'+"="*10)
    print(set_zero_fe)
    print("="*10+'_set_zero_fe_2_'+"="*10)
    print(set_zero_fe_2)
    print("="*10+'_set_zero_fe_3_'+"="*10)
    print(set_zero_fe_3)
    print("="*10+'_set_zero_fe_4_'+"="*10)
    print(set_zero_fe_4)
    print("="*10+'_set_zero_fe_5_'+"="*10)
    print(set_zero_fe_5)
    print("="*10+'_set_zero_fe_6_'+"="*10)
    print(set_zero_fe_6)
    print("="*10+'_set_zero_fe_7_'+"="*10)
    print(set_zero_fe_7)
#     print("="*10+'_set_near_fe_'+"="*10)
#     print(set_near_fe)
    for j in fe:
        if j not in me_fe + cat_fe + bool_fe + set_zero_fe + set_zero_fe_2 + set_zero_fe_3 + set_zero_fe_4+ set_zero_fe_5+ set_zero_fe_6+ set_zero_fe_7:
            """填充数据"""
            data[j] = data[j].interpolate(method='nearest')
            
        elif j in set_zero_fe:
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest',
                                limit_direction = 'both' ,limit = 3 )

            data[j] = data[j].fillna(data[j].mode()[0])
        elif j in  set_zero_fe_2 :
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest',
                                limit_direction = 'both' ,limit = 4 )

            data[j] = data[j].fillna(data[j].mode()[0])
            
        elif j in  set_zero_fe_3 :
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest',
                                limit_direction = 'both' ,limit = 5 )

            data[j] = data[j].fillna(data[j].mode()[0])
            
        elif j in set_zero_fe_4:
            
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest',
                                limit_direction = 'both' ,limit = 6 )
            data[j] = data[j].fillna(data[j].mode()[0])
            
            
        elif j in set_zero_fe_5:
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest',
                                limit_direction = 'both' ,limit = 8 )
            data[j] = data[j].fillna(data[j].mode()[0])
            
        elif j in set_zero_fe_6 + ['var019','var065','var059']:
#             data[j] = resample_fe_mode_roll(data,j,i,Type = 'mode',w = 11 , mw = 5).values
            data[j] = resample_fe_mode(data,j,i,Type = 'mode').values
            data[j] = data[j].interpolate(method='nearest')
            
        elif j in set_zero_fe_7 : 
            data[j] = data[j].rolling(window = 10 , min_periods = 5, center=True,
                                      closed = 'both').median()
            data[j] = data[j].interpolate(method='nearest')
            
        elif j in  ['var046','var042','var068','var034','var004','var043','var056','var031']:
            data[j] = data[j].rolling(window = 10 , min_periods = 5, center=True,
                                      closed = 'both').median()
#             data[j] = tranfer_fe(data,j,i).values
            data[j] = data[j].interpolate(method='nearest')
#         elif j in ['var027']:
#             data[j] = data[j].rolling(window = 35 , min_periods = 17, center=True,
#                           closed = 'both').mean()
#             data[j] = data[j].interpolate(method='time',limit = 167 )
#             data[j] = data[j].fillna(data[j].mode()[0])
        elif j in ['var027']:
            data[j] = data[j].rolling(window = 45 , min_periods = 30, center=True,
                          closed = 'both').median()
            data[j] = data[j].interpolate(method='nearest')
            
        elif j in me_fe:
            data[j] = data[j].rolling(window = 10 , min_periods = 5, center=True,
                                      closed = 'both').mean()
            data[j] = data[j].interpolate(method='time')
    
#         elif j in cat_fe:
#             data[j] = resample_fe_mode_1(data,j,i,Type = 'mode1').values
#             data[j] = data[j].interpolate(method='nearest',
#                                 limit_direction = 'both')
#         elif j in ['var053']:
#             data[j] = data[j].rolling(window = 10 , min_periods = 5, center=True,
#                                       closed = 'both').mean()
#             data[j] = list(map(lambda x:1 if x>0.2 else x,data[j]))
#             data[j] = data[j].interpolate(method='nearest',
#                                             limit_direction = 'both')
#         elif j in bool_fe:
#             data[j] = data[j].interpolate(method='nearest',
#                                 limit_direction = 'both')


    df = pd.concat([df,data],axis = 0)

  0%|          | 0/33 [00:00<?, ?it/s]

Time used: 3.252268
==========_1_==========
{'var066', 'var047', 'var016', 'var020', 'var053'}
==========_set_zero_fe_==========
['var008', 'var010', 'var013', 'var025', 'var026', 'var050']
==========_set_zero_fe_2_==========
['var032', 'var064']
==========_set_zero_fe_3_==========
['var009', 'var058']
==========_set_zero_fe_4_==========
['var023', 'var041', 'var049', 'var063']
==========_set_zero_fe_5_==========
['var017', 'var039', 'var048', 'var054', 'var060']
==========_set_zero_fe_6_==========
['var044']
==========_set_zero_fe_7_==========
['var015', 'var019', 'var043', 'var059', 'var065', 'var068']


  3%|▎         | 1/33 [00:06<03:23,  6.36s/it]

Time used: 3.190348
==========_1_==========
{'var066', 'var047', 'var016', 'var020', 'var053'}
==========_set_zero_fe_==========
['var008', 'var013', 'var025', 'var026', 'var032']
==========_set_zero_fe_2_==========
['var023', 'var044', 'var050']
==========_set_zero_fe_3_==========
['var048']
==========_set_zero_fe_4_==========
['var009', 'var010', 'var017', 'var041', 'var049', 'var058', 'var064']
==========_set_zero_fe_5_==========
['var039', 'var054', 'var056', 'var060', 'var063']
==========_set_zero_fe_6_==========
[]
==========_set_zero_fe_7_==========
['var019', 'var043', 'var059', 'var065', 'var068']


  6%|▌         | 2/33 [00:12<03:15,  6.31s/it]

...

 94%|█████████▍| 31/33 [04:09<00:18,  9.38s/it]

Time used: 4.430219999999991
==========_1_==========
{'var066', 'var047', 'var016', 'var020', 'var053'}
==========_set_zero_fe_==========
['var008', 'var025', 'var026', 'var032']
==========_set_zero_fe_2_==========
['var039', 'var049', 'var050']
==========_set_zero_fe_3_==========
['var009', 'var013', 'var041', 'var048', 'var064']
==========_set_zero_fe_4_==========
['var023', 'var058', 'var063']
==========_set_zero_fe_5_==========
['var010', 'var017', 'var054', 'var060']
==========_set_zero_fe_6_==========
['var044']
==========_set_zero_fe_7_==========
['var015', 'var019', 'var043', 'var059', 'var065', 'var068']


 97%|█████████▋| 32/33 [04:24<00:11, 11.06s/it]

Time used: 3.601968999999997
==========_1_==========
{'var066', 'var047', 'var016', 'var020', 'var053'}
==========_set_zero_fe_==========
['var008', 'var025', 'var026', 'var032']
==========_set_zero_fe_2_==========
['var050', 'var064']
==========_set_zero_fe_3_==========
['var013', 'var039', 'var048', 'var063']
==========_set_zero_fe_4_==========
['var009', 'var041', 'var049', 'var058']
==========_set_zero_fe_5_==========
['var017', 'var023', 'var054', 'var060']
==========_set_zero_fe_6_==========
['var010', 'var044']
==========_set_zero_fe_7_==========
['var019', 'var043', 'var059', 'var065']


100%|██████████| 33/33 [04:33<00:00, 10.47s/it]


In [14]:
sub = df[df.flag==1].copy().reset_index(drop = True)
del sub['flag']
res = pd.read_csv('../data/template_submit_result.csv',parse_dates=[0])[['ts','wtid']]
DF = res.merge(sub, on=['wtid','ts'],how = 'outer')

In [15]:
# DF[cat_fe+bool_fe] = DF[cat_fe+bool_fe].astype(int)
DF[cat_fe+bool_fe] = -1

In [16]:
"""人为调整"""
DF['var041'] = list(map(lambda x: 0 if abs(x)>=0.04 else x,DF['var041']))
DF['var058'] = list(map(lambda x: -abs(x) if abs(x)>=0.06 else x,DF['var058']))
DF['var058'] = list(map(lambda x: 0 if x >=0.03 else x,DF['var058']))

In [17]:
DF.iloc[:,2:] = DF.iloc[:,2:].round(2)

In [27]:
DF.to_csv('../submit/nn_v3.csv',index = False)